In [2]:
from __future__ import print_function
from __future__ import division
from __future__ import unicode_literals

import warnings
#import wandb
import os
import numpy as np
import deepchem as dc
import time
import csv
import tensorflow as tf
import pandas as pd
from deepchem.data import DiskDataset
from pandas import Series,DataFrame
from sklearn.metrics import accuracy_score,roc_auc_score
from deepchem.molnet import load_tox33
from deepchem.molnet import load_tox33_2
from deepchem.molnet import load_tox33_w
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.kernel_ridge import KernelRidge
warnings.filterwarnings('ignore')
os.environ['TF_XLA_FLAGS'] = '–tf_xla_enable_xla_devices'

In [ ]:

"""选择分子特征"""

#图像特征
g_feat = dc.feat.ConvMolFeaturizer(per_atom_fragmentation=False)()
g_feat = dc.feat.WeaveFeaturizer()
g_feat = dc.feat.MolGanFeaturizer()
g_feat = dc.feat.MolGraphConvFeaturizer(use_edges=True)

#数组特征
feat = dc.feat.MACCSKeysFingerprint()
feat = dc.feat.CircularFingerprint(size=2048, radius=4)
feat = dc.feat.RDKitDescriptors()
feat = dc.feat.MordredDescriptors()
feat = dc.feat.OneHotFeaturizer()
feat = dc.feat.SmilesToImage(img_size=80, img_spec='std')

splitters = {
    'index': dc.splits.IndexSplitter(),
    'random': dc.splits.RandomSplitter(),
    'scaffold': dc.splits.ScaffoldSplitter(),
    'butina': dc.splits.ButinaSplitter(),
    'fingerprint': dc.splits.FingerprintSplitter(),
    'task': dc.splits.TaskSplitter(),
    'stratified': dc.splits.RandomStratifiedSplitter()
}

transformers = {
    'balancing':
    TransformerGenerator(dc.trans.BalancingTransformer),
    'normalization':
    TransformerGenerator(dc.trans.NormalizationTransformer, transform_y=True),
    'minmax':
    TransformerGenerator(dc.trans.MinMaxTransformer, transform_y=True),
    'clipping':
    TransformerGenerator(dc.trans.ClippingTransformer, transform_y=True),
    'log':
    TransformerGenerator(dc.trans.LogTransformer, transform_y=True)
}
# Install wandb in shell
#pip install wandb

# Login in shell (required only once)
wandb login
# # Login in notebook (required only once)
import wandb
wandb.login()

# # Initialize a WandbLogger
logger = WandbLogger(…)

# Set `wandb_logger` when creating `KerasModel`
import deepchem as dc
# Log training loss to wandb
model = dc.models.KerasModel(…, wandb_logger=logger)
model.fit(…)

# Log validation metrics to wandb using ValidationCallback
import deepchem as dc
vc = dc.models.ValidationCallback(…)
model = KerasModel(…, wandb_logger=logger)
model.fit(…, callbacks=[vc])
logger.finish()
wandb.watch(model)#pytorch模型可用

In [4]:
"""" 一.加载数据集和表征数据集"""
feat = dc.feat.CircularFingerprint()
#split = dc.splits.RandomStratifiedSplitter()
split = dc.splits.FingerprintSplitter()
tox_tasks, tox_datasets, transformers = load_tox33(featurizer=feat, splitter= split)
train_dataset, valid_dataset, test_dataset = tox_datasets

In [5]:
train_dataset

<DiskDataset X.shape: (34939, 2048), y.shape: (34939, 31), w.shape: (34939, 31), task_names: ['Carcinogenicity' 'Ames Mutagenicity' 'Respiratory toxicity' ... 'BCF'
 'LC50' 'IGC50']>

In [6]:
test_dataset.to_dataframe()

,X1,X2,X3,X4,X5,X6,X7,X8,X9,X10,...,w23,w24,w25,w26,w27,w28,w29,w30,w31,ids
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,c1ccc(-c2ccccc2)cc1
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,c1ccccc1
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,c1ccc([SnH](c2ccccc2)c2ccccc2)cc1
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,c1ccc(-c2cccc(-c3ccccc3)c2)cc1
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,c1ccsc1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4363,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Nc1ccc(N)c2c1C(=O)c1ccccc1C2=O
4364,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Nc1ccc(N)c2ccccc12
4365,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,CNc1cccc2c1C(=O)c1ccccc1C2=O
4366,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Oc1cccc2c1ccc1c3ccccc3ccc21


In [15]:
"""二.定义模型可迭代的参数集合"""
params_dict = {
    'n_estimators': [500,1000,1500]
}

"""三.寻找最优参数集合"""
# optimizer = dc.hyper.GridHyperparamOpt(model)
# metric = dc.metrics.Metric(dc.metrics.roc_auc_score)
# best_model, best_hyperparams, all_results = optimizer.hyperparam_search(params_dict, 
#                                                                         train_dataset, 
#                                                                         valid_dataset, 
#                                                                         metric, 
#                                                                         transformers)

'三.寻找最优参数集合'

In [16]:
'''查看最优参数'''
# print(best_hyperparams)
# print("\n")
# all_results

'查看最优参数'

In [17]:
""""四.定义可视化模型Wandb及模型参数"""
# Initialize wandb and save hyperparameters
# wandb.init(
#   project="rf",
#   config={
#     'n_tasks': [len(tox_tasks)],
#     'n_estimators': 500
#     }
# )
# logger = wandb.config


'"四.定义可视化模型Wandb及模型参数'

In [18]:
# import tempfile

# with tempfile.TemporaryDirectory() as data_dir:
#     disk_train_dataset = dc.data.DiskDataset.from_numpy(train_dataset.X,train_dataset.y,train_dataset.w,train_dataset.get_task_names())
#     print(disk_train_dataset)

In [ ]:
"""五.建立模型，并寻找最优参数的周期"""
#设置必要的模型参数
# batch_size= 64,
# n_filters=128,
# n_fully_connected_nodes=128,
# seed=123
#learning_rate = dc.models.optimizers.ExponentialDecay(0.0001, 0.8, 1000)
'''定义评价指标'''
#metric = dc.metrics.Metric(dc.metrics.roc_auc_score, dc.metrics.precision_recall_curve,dc.metrics.accuracy_score,np.mean)
metric = dc.metrics.Metric(dc.metrics.roc_auc_score, np.mean)
'''定义损失函数'''
#loss_fuc = dc.models.losses.SigmoidCrossEntropy
'''定义模型参数'''

In [ ]:
#RFmodel
def model_builder(model_dir):
    sklearn_model = RandomForestClassifier(class_weight="balanced", n_estimators= 500)
    return dc.models.sklearn_models.SklearnModel(sklearn_model, model_dir)

model = dc.models.multitask.SingletaskToMultitask(tox_tasks,model_builder)
model.fit(train_dataset)

In [ ]:
# Building scikit learn Kernel SVM model
# from concurrent.futures import ThreadPoolExecutor,ProcessPoolExecutor
# import time
def model_builder(model_dir):
    sklearn_model = SVC(C=1.0, gamma=0.05, class_weight="balanced", probability=True)
    return dc.models.SklearnModel(sklearn_model, model_dir)

model = dc.models.multitask.SingletaskToMultitask(tox_tasks, model_builder)

# with ProcessPoolExecutor(max_workers=16) as executor:
#     future = executor.submit(model.fit(train_dataset))
#executor.shutdown()
model.fit(train_dataset)

In [5]:

'''打印模型评估分数'''
metric = dc.metrics.Metric(dc.metrics.roc_auc_score, np.mean)
print("Evaluating model")
train_scores = model.evaluate(train_dataset, [metric], transformers)
valid_scores = model.evaluate(valid_dataset, [metric], transformers)
#test_scores = model.evaluate(test_dataset, [metric], transformers)

print("Train scores")
print(train_scores)

print("Validation scores")
print(valid_scores)

# print("test scores")
# print(test_scores)

Evaluating model
Train scores
{'mean-roc_auc_score': 0.9636458780586146}
Validation scores
{'mean-roc_auc_score': 0.6764371609949403}


In [6]:

'''打印模型评估分数'''
metric = dc.metrics.Metric(dc.metrics.roc_auc_score, np.mean)
print("Evaluating model")
train_scores = model.evaluate(train_dataset, [metric], transformers)
valid_scores = model.evaluate(valid_dataset, [metric], transformers)
test_scores = model.evaluate(test_dataset, [metric], transformers)

print("Train scores")
print(train_scores)

print("Validation scores")
print(valid_scores)

print("test scores")
print(test_scores)

Evaluating model
Train scores
{'mean-roc_auc_score': 0.9636458780586146}
Validation scores
{'mean-roc_auc_score': 0.6764371609949403}
test scores
{'mean-roc_auc_score': 0.683295902976739}


In [ ]:
"""加载外部数据集"""
dataset_file = "/home/deepchem/datasets/Hepatotoxicity.csv"


task = ['Hepatotoxicity']
g_feat = dc.feat.ConvMolFeaturizer()

loader = dc.data.data_loader.CSVLoader(tasks=task, smiles_field="smiles", featurizer=g_feat)
dataset = loader.create_dataset(dataset_file)

transformer = transformers
dataset = transformer.transform(dataset)
dataset

In [ ]:
"""预测"""
'''预测外部数据集'''
# pred_task = tox21_tasks
# pred_file = "./test_datasets/TMC_ZINC_35308_CSV - 副本.csv"
# graph_featurizer = dc.feat.graph_features.ConvMolFeaturizer()
# pred_loader = dc.data.data_loader.CSVLoader(tasks=pred_task, smiles_field="smiles", featurizer=graph_featurizer)
# pred_dataset_featurized = pred_loader.featurize(pred_file)
'''预测并转换为Dataframe 格式'''
predict = model.predict(dataset,transformers)
print(predict.shape)
# pred_scores = model.evaluate(dataset, [metric], transformers)
# print(pred_scores)
# df = pd.DataFrame.from_records(predict,index=dataset.ids)
# df.insert(0,'labels',dataset.y)
# df.to_csv("/home/deepchem/pred/pred_file.csv")
'''计算预测结果的AUC值'''

# y_pred= predict[:,:,1].reshape(1313)
# y_true= dataset.y.reshape(1313)
# roc_auc_score(y_true, y_pred)

In [ ]:
y_pred= predict[:,:,1].reshape(1313)
y_pred

y_true= dataset.y.reshape(1313)
y_true

In [ ]:
"""精确度评分数据处理"""
a = []
for i in y_pred:
     #print(i)
    if i > 0.5:
        i = 1;
    else:
        i = 0;
    a.append(i)
y_pred_int=np.array(a)
met = dc.metrics.accuracy_score(y_true, y_pred_int)
met

In [ ]:
b = np.array(a)
type(b)
accuracy_score(y_true, b)